<a href="https://colab.research.google.com/github/KuzmenkoO/amazinum_home_work/blob/main/HW_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np


def load_energy_data():
    # Завантажуємо дані про енергетику з файлу
    file_name_energy= 'Energy Indicators.xls'
    energy = pd.read_excel(file_name_energy, skiprows=17, skipfooter=38, usecols="C:F")

    # Перейменовуємо стовпці
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    # Замінюємо '...' на NaN
    pd.set_option('future.no_silent_downcasting', True)
    energy = energy.replace("...", np.nan).infer_objects(copy=False)

    # Конвертуємо Energy Supply з петаджоулів у гігаджоулі
    energy['Energy Supply'] *= 1000000

    # Очищуємо назви країн від чисел і дужок
    energy['Country'] = energy['Country'].str.replace(r"\d+", "", regex=True)
    energy['Country'] = energy['Country'].str.replace(r" \(.*\)", "", regex=True)

    # Перейменовуємо країни
    country_renames = {
        "Republic of Korea": "South Korea",
        "United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong"
    }
    energy['Country'] = energy['Country'].replace(country_renames)

    return energy


def load_gdp_data():
    # Завантажуємо дані про ВВП
    file_name_gdp = 'world_bank.csv'
    GDP = pd.read_csv(file_name_gdp, skiprows=4)

    # Перейменовуємо країни
    country_renames = {
        "Korea, Rep.": "South Korea",
        "Iran, Islamic Rep.": "Iran",
        "Hong Kong SAR, China": "Hong Kong"
    }
    GDP.loc[:, 'Country Name'] = GDP['Country Name'].replace(country_renames)

    # Вибираємо лише необхідні стовпці
    columns_to_keep = ['Country Name'] + [str(year) for year in range(2006, 2016)]
    GDP = GDP[columns_to_keep]

    GDP.rename(columns={'Country Name': 'Country'}, inplace=True)

    return GDP


def load_scimen_data():
    # Завантажуємо дані про наукові публікації
    file_name_scimEn = "scimagojr-3.xlsx"
    ScimEn = pd.read_excel(file_name_scimEn)

    return ScimEn


def answer_one():
    energy = load_energy_data()
    GDP = load_gdp_data()
    ScimEn = load_scimen_data()

    # Об'єднуємо всі три таблиці за спільними країнами
    merged_df = pd.merge(ScimEn, energy, on='Country', how='inner')
    merged_df = pd.merge(merged_df, GDP, on='Country', how='inner')

    # Відбираємо лише топ-15 країн за рангом
    merged_df = merged_df[merged_df['Rank'] <= 15]

    merged_df = merged_df[['Country'] + ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
                         'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita',
                         '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    # Встановлюємо індекс на Country
    merged_df.set_index('Country', inplace=True)

    return merged_df


def answer_two():
    Top15 = answer_one()
    avgGDP = Top15.loc[:, '2006':'2015'].mean(axis=1).sort_values(ascending=False)
    return avgGDP


def answer_three():
    Top15 = answer_one()
    avgGDP = answer_two()
    country = avgGDP.index[5]  # 6-та країна за середнім ВВП
    GDP_change = Top15.loc[country, '2015'] - Top15.loc[country, '2006']
    return GDP_change


def answer_four():
    Top15 = answer_one()
    Top15['Self-Citation Ratio'] = Top15['Self-citations'] / Top15['Citations']
    max_country = Top15['Self-Citation Ratio'].idxmax()
    max_value = float(Top15['Self-Citation Ratio'].max())
    return (max_country, max_value)


def answer_five():
    Top15 = answer_one()
    Top15['Population Estimate'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15 = Top15.sort_values(by='Population Estimate', ascending=False)
    return Top15.index[2]


def answer_six():
    Top15 = answer_one()
    Top15['Population Estimate'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per capita'] = Top15['Citable documents'] / Top15['Population Estimate']
    correlation = Top15['Citable docs per capita'].corr(Top15['Energy Supply per Capita'])
    return correlation


def answer_seven():
    Top15 = answer_one()
    Top15['Population Estimate'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']

    ContinentDict = {'China': 'Asia',
                     'United States': 'North America',
                     'Japan': 'Asia',
                     'United Kingdom': 'Europe',
                     'Russian Federation': 'Europe',
                     'Canada': 'North America',
                     'Germany': 'Europe',
                     'India': 'Asia',
                     'France': 'Europe',
                     'South Korea': 'Asia',
                     'Italy': 'Europe',
                     'Spain': 'Europe',
                     'Iran': 'Asia',
                     'Australia': 'Australia',
                     'Brazil': 'South America'}

    Top15['Continent'] = Top15.index.to_series().map(ContinentDict)
    summary_df = Top15.groupby('Continent')['Population Estimate'].agg(['size', 'sum', 'mean', 'std'])

    return summary_df

def dividing_line(a):
    a = str(a) + " "
    print("\nQuestion №" + a + "_____________" * 5)


dividing_line(1)
print(answer_one())
dividing_line(2)
print(answer_two())
dividing_line(3)
print(answer_three())
dividing_line(4)
print(answer_four())
dividing_line(5)
print(answer_five())
dividing_line(6)
print(answer_six())
dividing_line(7)
print(answer_seven())


Question №1 _________________________________________________________________
                    Rank  Documents  Citable documents  Citations  \
Country                                                             
China                  1     402188             400158    5077232   
United States          2     208042             203356    3540288   
India                  3      81853              79757     968523   
Japan                  4      58342              57735     717946   
United Kingdom         5      56288              54713    1095371   
Germany                6      50906              49773     777362   
Russian Federation     7      46186              45868     217996   
Canada                 8      41209              40390     915491   
Italy                  9      38700              36909     639473   
South Korea           10      38362              38046     618762   
Iran                  11      32080              31725     634135   
France                12